In [0]:
from pyspark.sql.utils import AnalysisException
import os



In [0]:
host = os.getenv('DB_HOST')
database = os.getenv('DB_DATABASE')
port = os.getenv('DB_PORT')
username = os.getenv('DB_USERNAME')
password = os.getenv('DB_PASSWORD')
url = f"jdbc:postgresql://{host}:{port}/{database}"

properties = {
    "user": username,
    "password": password,
    "driver": "org.postgresql.Driver"
}


In [0]:

tables_query = "(\
    SELECT \
        table_name \
    FROM \
        information_schema.tables \
    WHERE table_schema = 'public' \
        and table_name not like 'pg_%' \
) AS table_list"
tables_df = spark.read.jdbc(url=url, table=tables_query, properties=properties)
table_names = [row['table_name'] for row in tables_df.collect()]

print(table_names)


['customers', 'employees', 'offices', 'orderdetails', 'orders', 'payments', 'product_lines', 'products']


In [0]:
for table in table_names:
    try:
        table_df = spark.read.jdbc(url=url, table=table, properties=properties)
        table_df = table_df.repartition(1)
        delta_path = f"/mnt/case/delta/{table}"  
        table_df.write.format("delta").mode("overwrite").save(delta_path)
        spark.sql(f"CREATE TABLE IF NOT EXISTS {table} USING DELTA LOCATION '{delta_path}'")
        print(f"Tabela {table} exportada para Delta Lake em {delta_path}")
        print(f"Tabela {table} registrada no catálogo com sucesso.")
    except AnalysisException as e:
        print(f"Erro ao exportar a tabela {table}: {e}")
    except Exception as e:
        print(f"Erro geral ao exportar a tabela {table}: {e}")

+---------------+--------------------+-----------------+------------------+-----------------+--------------------+--------------------+-------------+--------+-----------+---------+-------------------------+------------+
|customer_number|       customer_name|contact_last_name|contact_first_name|            phone|       address_line1|       address_line2|         city|   state|postal_code|  country|sales_rep_employee_number|credit_limit|
+---------------+--------------------+-----------------+------------------+-----------------+--------------------+--------------------+-------------+--------+-----------+---------+-------------------------+------------+
|            103|                Jake|             King|           Carine |       40.32.2555|      54, rue Royale|                null|       Nantes|Victoria|      44000|   France|                     1370|    21000.00|
|            112|   Signal Gift Store|             King|              Jean|       7025551838|     8489 Strong St.|      